In [1]:
!pip3 freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd

In [3]:
year = 2021
month = 2
output_file = f'predictions_{year:04d}_{month:02d}.parquet'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [8]:
y_pred.mean()

16.191691679979066

In [9]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [11]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
!ls -lh

total 87680
-rw-r--r--  1 mikhail  staff    72B 24 июн 22:46 Dockerfile
-rw-r--r--  1 mikhail  staff   190B 25 июн 00:17 Pipfile
-rw-r--r--  1 mikhail  staff    14K 25 июн 00:19 Pipfile.lock
-rw-r--r--  1 mikhail  staff    17K 24 июн 22:46 model.bin
-rw-r--r--  1 mikhail  staff    19M 25 июн 00:37 predictions_2021_02.parquet
-rw-r--r--  1 mikhail  staff    24M 25 июн 00:29 predictions_2021_03.parquet
-rw-r--r--  1 mikhail  staff   4,6K 25 июн 00:37 starter.ipynb
-rw-r--r--  1 mikhail  staff   1,2K 25 июн 00:30 starter.py
